# Send Email: Python으로 email 보내기
- 자동화에 유용하게 사용할 수 있는 파이썬 라이브러리를 배워보자. python으로 이메일 보내기!
    - e.g. 광고메일 등을 보낼 수 있음
    - e.g. 머신러닝 모델을 학습시켜놓고 결과 데이터를 csv 파일로 메일을 받아볼 수 있음

# 1. Preparation

파이썬으로 이메일을 보내기 위한 준비를 해준다.

## 1.1 Importing necessary libraries

필요한 패키지들을 import해준다.

In [1]:
import smtplib, os, pickle  # smtplib: 메일 전송을 위한 패키지
from email import encoders  # 파일전송을 할 때 이미지나 문서 동영상 등의 파일을 문자열로 변환할 때 사용할 패키지
from email.mime.text import MIMEText   # 본문내용을 전송할 때 사용되는 모듈
from email.mime.multipart import MIMEMultipart   # 메시지를 보낼 때 메시지에 대한 모듈
from email.mime.base import MIMEBase     # 파일을 전송할 때 사용되는 모듈

메일을 보낼 주소들을 미리 저장해준다.  
테스트를 위해 여러가지 주소로 보내고 싶은데, 나는 계정을 여러 개 갖고 있지 않으므로, 
내 이메일 주소인 hyeshinoh@gmail.com에서 hyeshinoh 뒤에 '+'와 아무 글자를 넣어주면 마치 여러 개의 이메일 주소인 것처럼 사용할 수 있다. 이렇게 쓰면 메일은 모두 hyeshinoh@gmail.com로 전송된다.

In [2]:
toAddr = ["panda706@naver.com", "hyeshinoh+11@gmail.com", "hyeshinoh+h@gmail.com"]

## 1.2 Setting login information: email & password
password는 직접 노출하면 유출의 위험이 있으므로 아래와 같이 pickle 파일로 저장해서 이용하는 것이 좋다.

In [4]:
# pw = "본인의 pw"    
# pickle.dump(pw, open("pw.pickle", 'wb'))

In [3]:
email = "hyeshinoh@gmail.com"
pw = pickle.load(open('pw.pickle', 'rb'))

# 2. Send text

## 2.1 Access smtp server

smtp object를 생성한다.

In [27]:
smtp = smtplib.SMTP('smtp.gmail.com', 587)   # 587: 서버의 포트번호
smtp.ehlo()
smtp.starttls()   # tls방식으로 접속, 그 포트번호가 587
# smtp.login('hyeshinoh@gmail.com', pickle.load( open('../pw.pickle', 'rb') ))
smtp.login('hyeshinoh@gmail.com', pw)

(235, b'2.7.0 Accepted')

## 2.2 Make message

msg object를 생성하고 제목을 넣어준다.

In [20]:
msg = MIMEMultipart()    #msg obj.
msg['Subject'] = 'hyeshin의 SMTP Send Text 테스트'

msg object에 본문(text msg)을 추가해준다.

In [21]:
# text msg
part = MIMEText('SMTP로 메일 보내기 본문 메시지입니다.')
msg.attach(part)   #msg에 part obj.를 추가해줌

In [22]:
msg

## 2.3 email 전송하기

In [23]:
for addr in toAddr:
    msg["To"] = addr
    smtp.sendmail("hyeshinoh@gmail.com", addr, msg.as_string())
        #msg는 object이기 때문에 전송을 위해 .as_string()으로 문자열로 바꿔야함(parsing)
    print(addr)

panda706@naver.com
hyeshinoh+11@gmail.com
hyeshinoh+h@gmail.com


메일이 다음과 같이 잘 도착했다!
- naver메일
![](https://github.com/hyeshinoh/Study_Web/blob/master/web_06_Send_Email_naver_mail_001.png?raw=true)
- gmail
![](https://github.com/hyeshinoh/Study_Web/blob/master/web_06_Send_Email_gmail_001.png?raw=true)

## 2.4 HTML code 추가하기

In [25]:
part_html = MIMEText('<br><a href="https://github.com/hyeshinoh/">hyeshin github</a>', 'html')
msg.attach(part_html)
msg  #msg 상태: 제목, 본문(text), html 코드

## 2.5 email 전송하기

In [28]:
# email 전송
for addr in toAddr:
    msg["To"] = addr
    smtp.sendmail("hyeshinoh@gmail.com", addr, msg.as_string())
    print(addr)

panda706@naver.com
hyeshinoh+11@gmail.com
hyeshinoh+h@gmail.com


# 3. Send File
이제 파일을 첨부해서 이메일을 보내보자

## 3.1 Access smtp server

smtp object를 다시 생성해보자.

In [29]:
# smtp close
smtp.quit()

(221, b'2.0.0 closing connection o68-v6sm26409410pfi.180 - gsmtp')

In [4]:
# Access smtp server
smtp = smtplib.SMTP('smtp.gmail.com', 587)   # 587: 서버의 포트번호
smtp.ehlo()
smtp.starttls()   
smtp.login('hyeshinoh@gmail.com', pw)

(235, b'2.7.0 Accepted')

## 3.2 Make message

msg object를 생성하고 제목을 넣어준다.

In [5]:
msg = MIMEMultipart()
msg['Subject'] = 'SMTP send file 테스트'

msg object에 본문(text msg)을 추가해준다.

In [6]:
part = MIMEText('SMTP로 send file 본문 메시지')
msg.attach(part)

파일을 첨부해보자. 단, javascript file은 보안 상 이유로 보낼 수 없게 되어 있음

In [7]:
# Attach FILE
path = 'apple.mp4'
# ctype = 'application/octet-stream'
# maintype, subtype = ctype.split('/', 1)
with open(path, 'rb') as f:
    part = MIMEBase("application", "octet-stream")
    part.set_payload(f.read())    #payload: osi 7-layers
    encoders.encode_base64(part)  #base64 encoding: 영상, 이미지 파일을 문자열 형태로 변환
    part.add_header('Content-Disposition', 'attachment', filename=path)
    msg.attach(part)
msg  # 제목, 본문, 파일

## 3.3 email 전송하기

In [8]:
for addr in toAddr:
    msg["To"] = addr
    smtp.sendmail("hyeshinoh@gmail.com", addr, msg.as_string())
    print(addr)

panda706@naver.com
hyeshinoh+11@gmail.com
hyeshinoh+h@gmail.com


메일이 다음과 같이 잘 도착했다!
- naver메일
![](https://github.com/hyeshinoh/Study_Web/blob/master/web_06_Send_Email_naver_mail_002.png?raw=true)
- gmail
![](https://github.com/hyeshinoh/Study_Web/blob/master/web_06_Send_Email_gmail_002.png?raw=true)

In [21]:
# smtp close
smtp.quit()

(221, b'2.0.0 closing connection d22-v6sm9509327pfk.69 - gsmtp')

#### 참고자료
- 패스트캠퍼스, ⟪데이터사이언스스쿨 8기⟫ 수업자료